In [ ]:
import os
import s3fs
import re
import pandas as pd
os.environ["AWS_ACCESS_KEY_ID"] = '1J9SZCURG0IZM0VVX37D'
os.environ["AWS_SECRET_ACCESS_KEY"] = '9CKCagFEDjzIKptrZy1sBOI2+C8+94ojH+LONi54'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIxSjlTWkNVUkcwSVpNMFZWWDM3RCIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzYxNjM5MDM3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzYyMjQzODYwLCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc2MTYzOTA2MCwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDo4NzNjZTViNy04ZDQ3LWQ0NGItNDk3NC04NWNjNDA3NWJkZWUiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6IjdlMzg4NmFiLWUxYWEtNDZiNS04MGE2LTRkNDRiYzk0NzJkZiIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.oBZKGdOLRwmoT9SUMM2H5RfIPMoP22e8cotzbdlOdtscV3mC6HL1vp72nuBvp6kMECQOlsOeqVY7rSfNnMm8Jw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])



us_list = fs.glob("s3://lab/yob????.txt")
df=pd.DataFrame()
for i in us_list :
    with fs.open(f"s3://{i}") as f:
        temp = pd.read_csv(f, header=None, names=['Name', 'Gender', 'Count'])
    df=pd.concat([df,temp], axis=0,ignore_index=True)

s3_path = "s3://lab/usgnd.csv"

df.to_csv(s3_path, index=False)



In [18]:
s3_path = "s3://lab/usgnd.csv"

df.to_csv(s3_path, index=False)

In [9]:
import asyncio
from playwright.async_api import async_playwright

URL = "https://japanese-names.info/first-names/gender/girl-name/start-with-a-girl"

async def scrape_names(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/118.0.5993.118 Safari/537.36"
        )
        page = await context.new_page()
        await page.goto(url, timeout=0)

        # Wait for at least one name to appear
        await page.wait_for_selector("ul.name-list li h3", timeout=10000)

        # Scroll to load all names
        previous_height = await page.evaluate("document.body.scrollHeight")
        while True:
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
            await page.wait_for_timeout(1500)  # wait for new content to load
            current_height = await page.evaluate("document.body.scrollHeight")
            if current_height == previous_height:
                break
            previous_height = current_height

        # Extract names with proper inner_text evaluation
        name_elements = await page.query_selector_all("ul.name-list li h3")
        names = []
        for el in name_elements:
            # Use evaluate to get fully rendered text
            text = await el.evaluate("(node) => node.textContent.trim()")
            if text:
                names.append(text)

        await browser.close()
        return names

async def main():
    names = await scrape_names(URL)
    print(f"Found {len(names)} names ✅")
    for name in names:
        print(name)

# In Jupyter
await main()


Found 60 names ✅
Akari
Ayaka
Arisa
Asana
Akina
Aki
Aika
Airi
Ayami
Akiri
Aoi
Ayana
Asuka
Ayuka
Ami
Akiho
Ayumi
Asaka
Akie
Akiko
Akane
Ayu
Aina
Arisu
Asami
Akika
Amika
Ayame
Ayuna
Aya
Aria
Ai
Arika
Amiri
Aimi
Ako
Akira
Akihi
Ayane
Asahi
Ayako
Asune
Anna
Arina
Asako
Ayano
Asa
Asuna
Akana
Ayuri
Aiko
Akiha
Ayae
Ayari
Azusa
Akino
Azumi
Amina
Akiyo
Aisa


In [12]:
import asyncio
import string
import pandas as pd
from playwright.async_api import async_playwright

BASE_URL = "https://japanese-names.info/first-names/gender/{gender}-name/start-with-{letter}"
GIRL = "girl"
BOY = "boy"

async def scrape_page(page, url):
    """Scrape one page and return list of names"""
    await page.goto(url, timeout=0)
    # Wait for names to load
    try:
        await page.wait_for_selector("ul.name-list li h3", timeout=5000)
    except:
        return []  # no names on this page

    # Scroll to ensure dynamic content loads
    previous_height = await page.evaluate("document.body.scrollHeight")
    while True:
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await page.wait_for_timeout(1000)
        current_height = await page.evaluate("document.body.scrollHeight")
        if current_height == previous_height:
            break
        previous_height = current_height

    # Extract names
    name_elements = await page.query_selector_all("ul.name-list li h3")
    names = [await el.evaluate("(node) => node.textContent.trim()") for el in name_elements if await el.evaluate("(node) => node.textContent.trim()")]
    return names

async def scrape_gender(gender):
    all_data = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/118.0.5993.118 Safari/537.36"
        )
        page = await context.new_page()

        for letter in ["a"] :#string.ascii_lowercase:
            page_num = 1
            while True:
                if gender == GIRL:
                    url = f"{BASE_URL.format(gender=gender, letter=letter)}/page/{page_num}/"
                else:  # boys have no page number on single-page
                    url = f"{BASE_URL.format(gender=gender, letter=letter)}" if page_num == 1 else f"{BASE_URL.format(gender=gender, letter=letter)}/page/{page_num}/"

                names = await scrape_page(page, url)
                if not names:
                    break  # stop if page has no names

                # Append data
                for name in names:
                    all_data.append({"letter": letter, "page": page_num, "name": name})

                page_num += 1

        await browser.close()

    return pd.DataFrame(all_data)

async def main():
    print("Scraping girl names...")
    girls_df = await scrape_gender(GIRL)
    print(f"Found {len(girls_df)} girl names ✅")

    print("Scraping boy names...")
    #boys_df = await scrape_gender(BOY)
    #print(f"Found {len(boys_df)} boy names ✅")

    # Save to CSV if needed
    #girls_df.to_csv("japanese_girl_names.csv", index=False)
    #boys_df.to_csv("japanese_boy_names.csv", index=False)

    return girls_df#, boys_df

# In Jupyter, use:
girls_df = await main()


Scraping girl names...
Found 748 girl names ✅
Scraping boy names...
